## 2. Perform KWIC Searches using `getout_of_text_3` on COCA, GloWbE, and EcoLexicon

- Author: [Etienne P Jacquot](mailto:etienne.jacquot@asc.upenn.edu) (11/14/2025)

### Corpora

Both COCA and GloWbE now use **consistent nested structure**:

- **`corpus_name='coca'`** (default): `{genre: {year_or_id: DataFrame}}`
  - Example: `coca['acad']['2009']` → DataFrame
  
- **`corpus_name='glowbe'`**: `{country_code: {file_id: DataFrame}}`
  - Example: `glowbe['us']['g19']` → DataFrame
  
- **`corpus_name='EcoLexicon'`**: 
  - Manual CSV downloads from Sketch-Engine EcoLexicon
________________

In [1]:
!pip install -U getout_of_text_3 -qqq

## Run Imports

To use the latest code changes (including reading COCA & GloWbE offline licensed files from English-Corpora.org), import the module after installation: 

In [2]:
import getout_of_text_3 as got3
import pandas as pd
import os
    
print("getout_of_text_3 version:",  got3.__version__)

/Users/ejacquot/Documents/Github/getout_of_text_3/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


getout_of_text_3 version: 0.4.10


## Read Local Corpus Data into Notebook

- the `corpus_name` parameter specifies which corpus format to use (`'coca'` or `'glowbe'`), as each have different directory structures and file naming conventions.

    > Note: The module will automatically fall back to the COCA formatting if an unrecognized corpus name is provided. If you are using a English-Corpora.org corpus that is not supported (i.e. have a different structure) please create an issue on GitHub bringing this to the attention of the maintainer

In [3]:
# Define data directory of your local English-Corpora.org offline files
data_directory = '../../data/english-corpora.org/'

In [4]:
# Load COCA corpus
coca_corpus = got3.read_corpus(dir_of_text_files='{}/coca/'.format(data_directory), 
                               corpus_name='coca')

COCA Genres:   0%|          | 0/8 [00:00<?, ?genre/s]

Processing genre: mag


COCA Genres:  12%|█▎        | 1/8 [00:01<00:08,  1.22s/genre]

Finished genre: mag (total files: 30)
Processing genre: web


COCA Genres:  25%|██▌       | 2/8 [00:02<00:07,  1.19s/genre]

...

Finished genre: blog (total files: 34)
Processing genre: fic


COCA Genres:  88%|████████▊ | 7/8 [00:08<00:01,  1.10s/genre]

Finished genre: fic (total files: 30)
Processing genre: tvm


COCA Genres: 100%|██████████| 8/8 [00:09<00:00,  1.15s/genre]

Finished genre: tvm (total files: 30)


In [5]:
# Load GloWbE corpus
glowbe_corpus = got3.read_corpus(dir_of_text_files='{}/glowbe/'.format(data_directory), 
                                 corpus_name='glowbe')

GloWbE Countries:   5%|▍         | 1/22 [00:00<00:07,  2.63country/s]

Finished country: hk (total files: 15)


GloWbE Countries:   9%|▉         | 2/22 [00:01<00:20,  1.01s/country]

Finished country: au (total files: 30)


GloWbE Countries:  14%|█▎        | 3/22 [00:03<00:22,  1.20s/country]

...

Finished country: ke (total files: 15)


GloWbE Countries:  95%|█████████▌| 21/22 [00:17<00:00,  1.35country/s]

Finished country: us (total files: 75)


GloWbE Countries: 100%|██████████| 22/22 [00:18<00:00,  1.21country/s]

Finished country: jm (total files: 15)


In [6]:
# Manually download CSVs from Sketch Engine and load into a format compatible with getout_of_text_3
ecolexicon_corpus = {}

for csv_file in [file for file in os.listdir('../../data/sketch-engine/') if file.endswith('.csv')]:
    
    # read your Sketch-Engine exported CSV file, skipping the first 4 rows of metadata
    df = pd.read_csv('../../data/sketch-engine/{}'.format(csv_file), skiprows=4)

    # add a 'text' column combining Left, KWIC, and Right context (i.e. the original concordance lines)
    df['text']= df['Left']+' '+df['KWIC']+' '+df['Right']

    # store the dataframe in the ecolexicon_corpus dictionary, using a placeholder genre value 'eco_text's
    ecolexicon_corpus[csv_file] = {'eco_text': df}

___________________
## Perform Word Count Totals for Each Corpus

This step is to verify that the corpora have been read in correctly by checking total word counts against known values from English-Corpora.org documentation.

- **COCA**: 1.1 billion words
- **GloWbE**: 1.9 billion words
- **EcoLexicon**: 23.1 million words (*ignoring given we do not have the full database locally*)

In [7]:
# Calculate total word count across all COCA genres and subkeys
def count_words_in_text(text):
    """Count words in a text string."""
    if not isinstance(text, str):
        return 0
    return len(text.split())

#
def calculate_total_word_count(corpus):
    total_word_count = 0
    start_time=pd.Timestamp.now()
    for genre, subkeys in corpus.items():
        for subkey, dataframe in subkeys.items():
            if isinstance(dataframe, pd.DataFrame) and 'text' in dataframe.columns:
                # Count words in all text entries for this subkey
                subkey_word_count = dataframe['text'].apply(count_words_in_text).sum()
                total_word_count += subkey_word_count

    end_time=pd.Timestamp.now()
    elapsed_time = end_time - start_time
    print(f"⏱️ Elapsed time: {elapsed_time}")
    print(f"🎯 TOTAL GLOWBE CORPUS: {total_word_count:,} words")
    return total_word_count

In [8]:
coca_word_count = calculate_total_word_count(coca_corpus)
glowbe_word_count = calculate_total_word_count(glowbe_corpus)
ecolexicon_word_count = 23100000

⏱️ Elapsed time: 0 days 00:00:30.843643
🎯 TOTAL GLOWBE CORPUS: 1,178,812,039 words
⏱️ Elapsed time: 0 days 00:00:58.183438
🎯 TOTAL GLOWBE CORPUS: 2,138,605,793 words


_____________________
## Running Query Searches Across English-Corpora `COCA`

- **Corpus of Contemporary American English (COCA)**
    - See the public corpus here: https://www.english-corpora.org/coca
    - Run for the terms:
        - `best system`
        - `best method`
        - `national system`
        - `industry standard`


In [9]:
keyword='best system'
corpus_name='coca'
best_system_kwic_coca = got3.search_keyword_corpus(
                keyword=keyword,
                db_dict=coca_corpus,
                case_sensitive=False,
                show_context=True,
                context_words=20,
                output='print',
                parallel=True,
                n_jobs=None
)
# drop empty results
best_system_kwic_coca = {k: v for k, v in best_system_kwic_coca.items() if v}
# get total count
best_system_total_kwic_coca = sum(len(v) for v in best_system_kwic_coca.values())


🔍 COCA Corpus Search: 'best system'
🚀 Using parallel processing with 9 processes...

📚 MAG_1993 :
------------------------------
  📝 Text 902: development process with the American Automobile Manufacturers Association ( AAMA ) . For consumers , it promises to provide the **best system** to date for choosing oil most appropriate for car engines , particularly newer models . <p> Essentially , the new
  📝 Text 1278: first 100 years of the new republic 's existence , a profound religious purpose permeated American education ( The One **Best System** , 1974 ) . Education without Christian prayer , the exposition and memorization of the Decalogue as well as explicitly
  📝 Text 1278: fellows John E. Chubb and Terry M. Moe , who do not agree that is resulted in the " one **best system** " ( Politics , Markets and America 's Schools , 1990 ) . In the critics ' view , the
  ✅ Found 3 occurrence(s) in mag_1993

📚 MAG_1992 :
------------------------------
  📝 Text 902: the engineering process ' ' 

In [10]:
keyword='best method'
corpus_name='coca'
best_method_kwic_coca = got3.search_keyword_corpus(
                keyword=keyword,
                db_dict=coca_corpus,
                case_sensitive=False,
                show_context=True,
                context_words=20,
                output='print',
                parallel=True,
                n_jobs=None
)
best_method_kwic_coca = {k: v for k, v in best_method_kwic_coca.items() if v}
best_method_total_kwic_coca = sum(len(v) for v in best_method_kwic_coca.values())


🔍 COCA Corpus Search: 'best method'
🚀 Using parallel processing with 9 processes...

📚 MAG_1993 :
------------------------------
  📝 Text 910: . Women 's fatal crashes often occur on slippery roads or during lane changes . True or false ? The **best method** for acquiring a bargain basement-priced car is to : A ) Solicit help from an independent auto broker ; B
  📝 Text 1222: chemicals using 4,000 pounds of chemical explosives in December 1991 . Manheim grants that blasting may indeed have been the **best method** for disposing of the unstable chemicals . But he criticizes NSF for not having produced an impact assessment before taking
  📝 Text 1352: charges and turned this way and that as lasers minutely examine how combustion works , its speed , and the **best method** of doing it . Eighty @ @ @ @ @ @ @ @ @ @ , located near a Bureau
  📝 Text 1492: In this way , data can be much more efficiently stored . <p> RLE , however , is not the **best method** for compressing a moving video file

In [11]:
keyword='national system'
corpus_name='coca'
national_system_kwic_coca = got3.search_keyword_corpus(
                keyword=keyword,
                db_dict=coca_corpus,
                case_sensitive=False,
                show_context=True,
                context_words=20,
                output='print',
                parallel=True,
                n_jobs=None
)
national_system_kwic_coca = {k: v for k, v in national_system_kwic_coca.items() if v}
national_system_total_kwic_coca = sum(len(v) for v in national_system_kwic_coca.values())


🔍 COCA Corpus Search: 'national system'
🚀 Using parallel processing with 9 processes...

📚 MAG_1993 :
------------------------------
  📝 Text 281: @ @ @ are the tree on which wealth grows , " List wrote in another book , called The **National System** of Political Economy . The tree which bears the fruit is of greater value than the fruit itself . ...
  📝 Text 281: , and in the broadest definitions of well-being , he might be better off . As List wrote in The **National System** of Political Economy , Between each individual and entire humanity , however , stands the NATION , with its special
  📝 Text 281: catalogue of MIT 's vast library system Wade found an entry for just a single volume by List , The **National System** of Political Economy , in an edition published in 1885 . When Wade finally obtained the book , he found
  📝 Text 1225: strategizing to ensure that this time it is coordinated with state ratification . We must prepare for phasing in a **national system** of child care

In [12]:
keyword='industry standard'
corpus_name='coca'
industry_standard_kwic_coca = got3.search_keyword_corpus(
                keyword=keyword,
                db_dict=coca_corpus,
                case_sensitive=False,
                show_context=True,
                context_words=20,
                output='print',
                parallel=True,
                n_jobs=None
)
industry_standard_kwic_coca = {k: v for k, v in industry_standard_kwic_coca.items() if v}
industry_standard_total_kwic_coca = sum(len(v) for v in industry_standard_kwic_coca.values())


🔍 COCA Corpus Search: 'industry standard'
🚀 Using parallel processing with 9 processes...

📚 MAG_1993 :
------------------------------
  📝 Text 12: not gouging anybody , " Knight says . " Our gross profits are around 39 percent , right on the **industry standard** . We make our profit on the volume . A country like Indonesia is converting from farm labor to semiskilled
  📝 Text 125: efficiencies but low dehumidification . Lower speeds reduce efficiency some what , but increase dehumidification . Although there is no **industry standard** for rating the effectiveness at removing moisture , most literature does list water removal in pints per hour , which
  📝 Text 917: " accidents , which now are so rare as not to show up in the injury statistics . A voluntary **industry standard** instituted in 1986 reduced tipover accidents by half , according to the CPSC . <p> The proportion of injuries related
  📝 Text 923: " says Povlacs . <p> " Triple-dipping " is Aladan @ @ @ @ @ @ @ @ @ @ **indus

_____________________
## Running Query Searches Across English-Corpora `GloWbE`

- **Corpus of Global Web-Based English (GloWbE)**
    - See the public corpus here: https://www.english-corpora.org/glowbe
    - Run for the terms:
        - `best system`
        - `best method`
        - `national system`
        - `industry standard`


In [13]:
keyword='best system'
corpus_name='glowbe'
best_system_kwic_glowbe = got3.search_keyword_corpus(
                keyword=keyword,
                db_dict=glowbe_corpus,
                case_sensitive=False,
                show_context=True,
                context_words=20,
                output='print',
                parallel=True,
                n_jobs=None
)
best_system_kwic_glowbe = {k: v for k, v in best_system_kwic_glowbe.items() if v}
best_system_total_kwic_glowbe = sum(len(v) for v in best_system_kwic_glowbe.values())

🔍 COCA Corpus Search: 'best system'
🚀 Using parallel processing with 9 processes...

📚 HK_G08 :
------------------------------
  ❌ No matches found in hk_g08

📚 HK_G09 :
------------------------------
  ❌ No matches found in hk_g09

📚 HK_G07 :
------------------------------
  ❌ No matches found in hk_g07

📚 HK_G06 :
------------------------------
  ❌ No matches found in hk_g06

📚 HK_G10 :
------------------------------
  📝 Text 1710: a convenient 90 degree angle . Sometimes a large piece will fall out when the part is bevelled . The **best system** is to use a double ' V ' or double ' U ' joint , but in many cases in
  ✅ Found 1 occurrence(s) in hk_g10

📚 HK_G04 :
------------------------------
  ❌ No matches found in hk_g04

📚 HK_G05 :
------------------------------
  ❌ No matches found in hk_g05

📚 HK_G01 :
------------------------------
  ❌ No matches found in hk_g01

📚 HK_G02 :
------------------------------
  📝 Text 1945: Communications Society ( 2006-2007 ) . He was a co-recipien

In [14]:
keyword='best method'
corpus_name='glowbe'
best_method_kwic_glowbe = got3.search_keyword_corpus(
                keyword=keyword,
                db_dict=glowbe_corpus,
                case_sensitive=False,
                show_context=True,
                context_words=20,
                output='print',
                parallel=True,
                n_jobs=None
)
best_method_kwic_glowbe = {k: v for k, v in best_method_kwic_glowbe.items() if v}
best_method_total_kwic_glowbe = sum(len(v) for v in best_method_kwic_glowbe.values())

🔍 COCA Corpus Search: 'best method'
🚀 Using parallel processing with 9 processes...

📚 HK_G08 :
------------------------------
  📝 Text 12: . I must think it out when I have more time . <p> 3 ) Have you thought out the **best method** ? <p> 4 ) All possible ways out of our difficulty have now been thought out . <p> 5. up
  📝 Text 739: is a loving and merciful . Mortal love from selfish started , possessive emotions certainly not long , so the **best method** is to put the love into the family . The west is because dopamine active and people have a strong
  📝 Text 873: one will always feel good and follow whatever the supplementary conditions will allow . <p> Buddhism teaches people that the **best method** to cultivate blessings is to get rid of selfishness . Selfishness can damage blessing and make people lonely . A
  📝 Text 2272: , our knowledge of the words and their meanings tends to become ' automatic ' Reading appears to be the **best method** for implicit learning - people who r

In [15]:
keyword='national system'
corpus_name='glowbe'
national_system_kwic_glowbe = got3.search_keyword_corpus(
                keyword=keyword,
                db_dict=glowbe_corpus,
                case_sensitive=False,
                show_context=True,
                context_words=20,
                output='print',
                parallel=True,
                n_jobs=None
)
national_system_kwic_glowbe = {k: v for k, v in national_system_kwic_glowbe.items() if v}
national_system_total_kwic_glowbe = sum(len(v) for v in national_system_kwic_glowbe.values())


🔍 COCA Corpus Search: 'national system'
🚀 Using parallel processing with 9 processes...

📚 HK_G08 :
------------------------------
  📝 Text 730: said , a system of national courts would be too costly . <p> John Rutledge of South Carolina opposed a **national system** of lower courts . But he argued for a national Supreme Court . <p> The convention voted for both .
  ✅ Found 1 occurrence(s) in hk_g08

📚 HK_G09 :
------------------------------
  📝 Text 1337: practice . To promote Scotland 's Earth heritage in an integrated manner to both geologists and non-specialists , a collaborative **national system** of ' hub and satellite'networks is proposed , using the Knockan Grag visitor ' hub ' and its Rock Route
  ✅ Found 1 occurrence(s) in hk_g09

📚 HK_G07 :
------------------------------
  📝 Text 914: 30 years this challenge has finally been met : a systematic approach has been adopted to plan and implement a **national system** of integrated surface transport -- road , rail , and water . <

In [16]:
keyword='industry standard'
corpus_name='glowbe'
industry_standard_kwic_glowbe = got3.search_keyword_corpus(
                keyword=keyword,
                db_dict=glowbe_corpus,
                case_sensitive=False,
                show_context=True,
                context_words=20,
                output='print',
                parallel=True,
                n_jobs=None
)
industry_standard_kwic_glowbe = {k: v for k, v in industry_standard_kwic_glowbe.items() if v}
industry_standard_total_kwic_glowbe = sum(len(v) for v in industry_standard_kwic_glowbe.values())


🔍 COCA Corpus Search: 'industry standard'
🚀 Using parallel processing with 9 processes...

📚 HK_G08 :
------------------------------
  📝 Text 1662: been focused on creating an organization that has a razor sharp focus on program specific results that provides donors with **industry standard** levels of transparency . With the support of an incredible group of employees , volunteers , and donors , The
  ✅ Found 1 occurrence(s) in hk_g08

📚 HK_G09 :
------------------------------
  📝 Text 37: was the Nikonos which began life as the Calypsophot , designed for Jacques Cousteau in 1956 . Once the professional **industry standard** , the Nikonos was a film camera which ultimately fell victim to the advent of high resolution digital cameras and
  📝 Text 279: copyright prices becoming more rational , and upstream and downstream cooperation with industry colleagues easier to establish and optimize the **industry standard** in the field of video marketing . Yu Chau last Tudou and Youku , two vid

_____________________
## Running Query Searches Across Sketch Engine `EcoLexicon-Corpus`

- See the public corpus here: https://www.sketchengine.eu/ecolexicon-corpus/
- Manually downloaded from Sketch Engine web interface as CSV files for the `EcoLexicon` corpus for the terms:
  - `best system`
  - `best method`
  - `national system`
  - `industry standard`


In [17]:
keyword='best system'
corpus_name='ecolexicon'
best_system_kwic_ecolexicon = got3.search_keyword_corpus(
                keyword=keyword,
                db_dict=ecolexicon_corpus,
                case_sensitive=False,
                show_context=True,
                context_words=20,
                output='print',
                parallel=True,
                n_jobs=None
)
best_system_kwic_ecolexicon = {k: v for k, v in best_system_kwic_ecolexicon.items() if v}
best_system_total_kwic_ecolexicon = sum(len(v) for v in best_system_kwic_ecolexicon.values())


🔍 COCA Corpus Search: 'best system'
🚀 Using parallel processing with 9 processes...

📚 CONCORDANCE_INDUSTRY_STANDARD_ECOLEXICON_EN_20251115000927.CSV_ECO_TEXT :
------------------------------
  ❌ No matches found in concordance_industry_standard_ecolexicon_en_20251115000927.csv_eco_text

📚 CONCORDANCE_BEST_SYSTEM_ECOLEXICON_EN_20251115000951.CSV_ECO_TEXT :
------------------------------
  📝 Text 0: The book aims to shed light on water quality issues such as how water quality is determined , what the **best system** of management measures are for achieving specified water quality , and what the most effective institutional or
  📝 Text 1: and to longshore currents. </s><s> (2) Summary. </s><s> In summary, referring to Table III-3-2, the CRAB emerges as the overall **best system** . </s><s> The sea sled provides slightly better overall vertical accuracy; however, as noted, the CRAB now utilizes an
  ✅ Found 2 occurrence(s) in concordance_best_system_ecolexicon_en_20251115000951.csv_eco_te

In [18]:
keyword='best method'
corpus_name='ecolexicon'
best_method_kwic_ecolexicon = got3.search_keyword_corpus(
                keyword=keyword,
                db_dict=ecolexicon_corpus,
                case_sensitive=False,
                show_context=True,
                context_words=20,
                output='print',
                parallel=True,
                n_jobs=None
)
best_method_kwic_ecolexicon = {k: v for k, v in best_method_kwic_ecolexicon.items() if v}
best_method_total_kwic_ecolexicon = sum(len(v) for v in best_method_kwic_ecolexicon.values())

🔍 COCA Corpus Search: 'best method'
🚀 Using parallel processing with 9 processes...

📚 CONCORDANCE_INDUSTRY_STANDARD_ECOLEXICON_EN_20251115000927.CSV_ECO_TEXT :
------------------------------
  ❌ No matches found in concordance_industry_standard_ecolexicon_en_20251115000927.csv_eco_text

📚 CONCORDANCE_BEST_SYSTEM_ECOLEXICON_EN_20251115000951.CSV_ECO_TEXT :
------------------------------
  ❌ No matches found in concordance_best_system_ecolexicon_en_20251115000951.csv_eco_text

📚 CONCORDANCE_BEST_METHOD_ECOLEXICON_EN_20251115001001.CSV_ECO_TEXT :
------------------------------
  📝 Text 0: predicted covariance structures was first assessed with a maximum-likelihood goodness-of-fit test , which is the **best method** for severe deviations from multivariate normality in rather small sample sizes ( Finch et al. , 1997 ). </s><s> The
  📝 Text 1: predicted covariance structures was first assessed with a maximum-likelihood goodness-of-fit test , which is the **best method** for severe deviation

In [19]:
keyword='national system'
corpus_name='ecolexicon'
national_system_kwic_ecolexicon = got3.search_keyword_corpus(
                keyword=keyword,
                db_dict=ecolexicon_corpus,
                case_sensitive=False,
                show_context=True,
                context_words=20,
                output='print',
                parallel=True,
                n_jobs=None
)
national_system_kwic_ecolexicon = {k: v for k, v in national_system_kwic_ecolexicon.items() if v}
national_system_total_kwic_ecolexicon = sum(len(v) for v in national_system_kwic_ecolexicon.values())

🔍 COCA Corpus Search: 'national system'
🚀 Using parallel processing with 9 processes...

📚 CONCORDANCE_INDUSTRY_STANDARD_ECOLEXICON_EN_20251115000927.CSV_ECO_TEXT :
------------------------------
  ❌ No matches found in concordance_industry_standard_ecolexicon_en_20251115000927.csv_eco_text

📚 CONCORDANCE_BEST_SYSTEM_ECOLEXICON_EN_20251115000951.CSV_ECO_TEXT :
------------------------------
  ❌ No matches found in concordance_best_system_ecolexicon_en_20251115000951.csv_eco_text

📚 CONCORDANCE_BEST_METHOD_ECOLEXICON_EN_20251115001001.CSV_ECO_TEXT :
------------------------------
  ❌ No matches found in concordance_best_method_ecolexicon_en_20251115001001.csv_eco_text

📚 CONCORDANCE_NATIONAL_SYSTEM_ECOLEXICON_EN_20251115001011.CSV_ECO_TEXT :
------------------------------
  📝 Text 4: the size of the harvest , with supply increasing continuously , was traced by the emergence and expansion of a **national system** of poor relief , also unique to England , '' affording the kind of provisio

In [20]:
keyword='industry standard'
corpus_name='ecolexicon'
industry_standard_kwic_ecolexicon = got3.search_keyword_corpus(
                keyword=keyword,
                db_dict=ecolexicon_corpus,
                case_sensitive=False,
                show_context=True,
                context_words=20,
                output='print',
                parallel=True,
                n_jobs=None
)
industry_standard_kwic_ecolexicon = {k: v for k, v in industry_standard_kwic_ecolexicon.items() if v}
industry_standard_total_kwic_ecolexicon = sum(len(v) for v in industry_standard_kwic_ecolexicon.values())

🔍 COCA Corpus Search: 'industry standard'
🚀 Using parallel processing with 9 processes...

📚 CONCORDANCE_INDUSTRY_STANDARD_ECOLEXICON_EN_20251115000927.CSV_ECO_TEXT :
------------------------------
  📝 Text 0: Ultimate on-the-Fly Network. </s><s> WIRED , December 2003. </s><s> J. S. Brown and D. Rejeski , Ecological Computing. </s><s> The **Industry Standard** , December 18 , 2000. </s><s> At : www. thestandard. com/article/0,1902,21365,00. html ? printer_friendly=. </s><s> Crossbow
  📝 Text 1: " ( see Figure 12. lb ). </s><s> Extensive environmental testing was done comparing it to TBTO , the current **industry standard** . </s><s> Sea-Nine " degraded very rapidl y with a half-life of 1day in seawater and 1 hour in sediment.
  📝 Text 3: relatively low-cost way to test the fit between a technology and the environment in which it must work. </s><s> Use **industry standard** technology. </s><s> Industry standards exist for almost every type of hardware , software , and communications tec

## Create Metadata DataFrame

Compile comprehensive statistics for KWIC results including:
- **Corpus**: Name of the corpus (COCA/GloWbE)
- **Total Genres**: Number of genres/categories in corpus
- **Total Word Count**: Total words in entire corpus
- **KWIC Genres**: Genres with keyword hits
- **KWIC Hits**: Total number of keyword occurrences
- **Hits per Million**: Normalized frequency (hits per 1M words)

In [21]:
def create_kwic_metadata(corpus_name, keyword_name, corpus_dict, kwic_results, total_word_count):
    """
    Create metadata row for KWIC search results.
    
    Args:
        corpus_name: Name of corpus (e.g., 'COCA', 'GloWbE')
        corpus_dict: Full corpus dictionary
        kwic_results: KWIC search results dictionary
        total_word_count: Total words in corpus
    
    Returns:
        Dictionary with metadata statistics
    """
    # Count total genres in corpus
    total_genres = len(corpus_dict.keys())
    
    # Count genres with KWIC hits
    kwic_genres = len(kwic_results.keys())
    
    # Count total KWIC hits
    total_kwic_hits = sum(len(v) for v in kwic_results.values())
    
    # Calculate hits per million words
    hits_per_million = (total_kwic_hits / total_word_count) * 1_000_000 if total_word_count > 0 else 0
    
    return {
        'Corpus': corpus_name,
        'Keyword': keyword_name,
        'Total Genres': total_genres,
        'Total Word Count': total_word_count,
        'KWIC Genres': kwic_genres,
        'KWIC Hits': total_kwic_hits,
        'Hits per Million': round(hits_per_million, 2)
    }


In [22]:
# Create metadata rows for both corpora
metadata_rows = []

######################################################

metadata_rows.append(create_kwic_metadata(
    corpus_name='COCA',
    keyword_name='best system',
    corpus_dict=coca_corpus,
    kwic_results=best_system_kwic_coca,
    total_word_count=coca_word_count
))

metadata_rows.append(create_kwic_metadata(
    corpus_name='GloWbE',
    keyword_name='best system',
    corpus_dict=glowbe_corpus,
    kwic_results=best_system_kwic_glowbe,
    total_word_count=glowbe_word_count
))

metadata_rows.append(create_kwic_metadata(
    corpus_name='EcoLexicon',
    keyword_name='best system',
    corpus_dict=ecolexicon_corpus,
    kwic_results=best_system_kwic_ecolexicon,
    total_word_count=2310000
))

######################################################

metadata_rows.append(create_kwic_metadata(
    corpus_name='COCA',
    keyword_name='best method',
    corpus_dict=coca_corpus,
    kwic_results=best_method_kwic_coca,
    total_word_count=coca_word_count
))

metadata_rows.append(create_kwic_metadata(
    corpus_name='GloWbE',
    keyword_name='best method',
    corpus_dict=glowbe_corpus,
    kwic_results=best_method_kwic_glowbe,
    total_word_count=glowbe_word_count
))

metadata_rows.append(create_kwic_metadata(
    corpus_name='EcoLexicon',
    keyword_name='best method',
    corpus_dict=ecolexicon_corpus,
    kwic_results=best_method_kwic_ecolexicon,
    total_word_count=2310000
))

######################################################

metadata_rows.append(create_kwic_metadata(
    corpus_name='COCA',
    keyword_name='national system',
    corpus_dict=coca_corpus,
    kwic_results=national_system_kwic_coca,
    total_word_count=coca_word_count
))

metadata_rows.append(create_kwic_metadata(
    corpus_name='GloWbE',
    keyword_name='national system',
    corpus_dict=glowbe_corpus,
    kwic_results=national_system_kwic_glowbe,
    total_word_count=glowbe_word_count
))

metadata_rows.append(create_kwic_metadata(
    corpus_name='EcoLexicon',
    keyword_name='national system',
    corpus_dict=ecolexicon_corpus,
    kwic_results=national_system_kwic_ecolexicon,
    total_word_count=2310000
))

######################################################

metadata_rows.append(create_kwic_metadata(
    corpus_name='COCA',
    keyword_name='industry standard',
    corpus_dict=coca_corpus,
    kwic_results=industry_standard_kwic_coca,
    total_word_count=coca_word_count
))

metadata_rows.append(create_kwic_metadata(
    corpus_name='GloWbE',
    keyword_name='industry standard',
    corpus_dict=glowbe_corpus,
    kwic_results=industry_standard_kwic_glowbe,
    total_word_count=glowbe_word_count
))

metadata_rows.append(create_kwic_metadata(
    corpus_name='EcoLexicon',
    keyword_name='industry standard',
    corpus_dict=ecolexicon_corpus,
    kwic_results=industry_standard_kwic_ecolexicon,
    total_word_count=2310000
))


In [23]:
import pandas as pd
metadata_df = pd.DataFrame(metadata_rows)
metadata_df.sort_values(by=['Corpus', 'Keyword'], inplace=True)
print(f"📊 Updated Metadata DataFrame created with {len(metadata_df)} rows")


📊 Updated Metadata DataFrame created with 12 rows


In [24]:
metadata_df['Total Word Count'] = metadata_df['Total Word Count'].astype(int)
metadata_df.style.format({
    'Total Word Count': '{:,}',
    'KWIC Hits': '{:,}',
    'Hits per Million': '{:.2f}'
})

,Corpus,Keyword,Total Genres,Total Word Count,KWIC Genres,KWIC Hits,Hits per Million
3,COCA,best method,8,"1,178,812,039",150,326,0.28
0,COCA,best system,8,"1,178,812,039",103,150,0.13
9,COCA,industry standard,8,"1,178,812,039",155,459,0.39
6,COCA,national system,8,"1,178,812,039",103,216,0.18
5,EcoLexicon,best method,4,"2,310,000",1,19,8.23
2,EcoLexicon,best system,4,"2,310,000",1,2,0.87
11,EcoLexicon,industry standard,4,"2,310,000",1,11,4.76
8,EcoLexicon,national system,4,"2,310,000",1,5,2.16
4,GloWbE,best method,20,"2,138,605,793",431,"1,332",0.62
1,GloWbE,best system,20,"2,138,605,793",250,416,0.19


In [25]:
metadata_df_temp = metadata_df[['Corpus', 'Keyword', 'Total Word Count', 
                                'KWIC Hits', 'Hits per Million']]
metadata_df_temp.style.format({
    'Total Word Count': '{:,}',
    'KWIC Hits': '{:,}',
    'Hits per Million': '{:.2f}'
})

,Corpus,Keyword,Total Word Count,KWIC Hits,Hits per Million
3,COCA,best method,"1,178,812,039",326,0.28
0,COCA,best system,"1,178,812,039",150,0.13
9,COCA,industry standard,"1,178,812,039",459,0.39
6,COCA,national system,"1,178,812,039",216,0.18
5,EcoLexicon,best method,"2,310,000",19,8.23
2,EcoLexicon,best system,"2,310,000",2,0.87
11,EcoLexicon,industry standard,"2,310,000",11,4.76
8,EcoLexicon,national system,"2,310,000",5,2.16
4,GloWbE,best method,"2,138,605,793","1,332",0.62
1,GloWbE,best system,"2,138,605,793",416,0.19


In [26]:
# Create a summary showing corpus metadata (non-duplicated)
corpus_summary = metadata_df.groupby('Corpus').agg({
    'Total Genres': 'first',
    'Total Word Count': 'first'
}).reset_index()

print("Corpus Summary (Total Word Counts):")
display(corpus_summary)

print("\nKWIC Hits by Keyword and Corpus:")
pivot_hits = metadata_df.pivot(index='Keyword', columns='Corpus', values='KWIC Hits')
display(pivot_hits)


Corpus Summary (Total Word Counts):


,Corpus,Total Genres,Total Word Count
0,COCA,8,1178812039
1,EcoLexicon,4,2310000
2,GloWbE,20,2138605793



KWIC Hits by Keyword and Corpus:


Corpus,COCA,EcoLexicon,GloWbE
Keyword,,,
best method,326,19,1332
best system,150,2,416
industry standard,459,11,1415
national system,216,5,511


___________________________
## Export KWIC results and metadata file to JSON

In [27]:
metadata_df.to_json('./exports/metadata_11142025_df.json')

In [28]:

from pathlib import Path
import json
import numpy as np
import pandas as _pd

# ensure exports directory exists
exports_dir = Path('./exports')
exports_dir.mkdir(parents=True, exist_ok=True)

def _serialize(obj):
    """Recursively convert common non-JSON-serializable objects to JSON-friendly types."""
    # numpy scalars / arrays
    if isinstance(obj, (np.integer,)):
        return int(obj)
    if isinstance(obj, (np.floating,)):
        return float(obj)
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    # pandas objects (pd imported earlier in the notebook)
    try:
        if isinstance(obj, _pd.Timestamp):
            return obj.isoformat()
        if isinstance(obj, _pd.DataFrame):
            return obj.to_dict(orient='records')
        if isinstance(obj, _pd.Series):
            return obj.tolist()
    except Exception:
        pass
    # Builtins
    if isinstance(obj, dict):
        return {k: _serialize(v) for k, v in obj.items()}
    if isinstance(obj, list):
        return [_serialize(v) for v in obj]
    if isinstance(obj, tuple):
        return [_serialize(v) for v in obj]
    # Fallback to string for unknown types
    return obj

# Serialize and dump KWIC results for each keyword and corpus combination
# COCA exports
kwic_coca_best_system = _serialize(best_system_kwic_coca)
kwic_coca_best_method = _serialize(best_method_kwic_coca)
kwic_coca_national_system = _serialize(national_system_kwic_coca)
kwic_coca_industry_standard = _serialize(industry_standard_kwic_coca)

with open(exports_dir / 'kwic_coca_best_system.json', 'w', encoding='utf-8') as f:
    json.dump(kwic_coca_best_system, f, ensure_ascii=False, indent=2)

with open(exports_dir / 'kwic_coca_best_method.json', 'w', encoding='utf-8') as f:
    json.dump(kwic_coca_best_method, f, ensure_ascii=False, indent=2)

with open(exports_dir / 'kwic_coca_national_system.json', 'w', encoding='utf-8') as f:
    json.dump(kwic_coca_national_system, f, ensure_ascii=False, indent=2)

with open(exports_dir / 'kwic_coca_industry_standard.json', 'w', encoding='utf-8') as f:
    json.dump(kwic_coca_industry_standard, f, ensure_ascii=False, indent=2)

# GloWbE exports
kwic_glowbe_best_system = _serialize(best_system_kwic_glowbe)
kwic_glowbe_best_method = _serialize(best_method_kwic_glowbe)
kwic_glowbe_national_system = _serialize(national_system_kwic_glowbe)
kwic_glowbe_industry_standard = _serialize(industry_standard_kwic_glowbe)

with open(exports_dir / 'kwic_glowbe_best_system.json', 'w', encoding='utf-8') as f:
    json.dump(kwic_glowbe_best_system, f, ensure_ascii=False, indent=2)

with open(exports_dir / 'kwic_glowbe_best_method.json', 'w', encoding='utf-8') as f:
    json.dump(kwic_glowbe_best_method, f, ensure_ascii=False, indent=2)

with open(exports_dir / 'kwic_glowbe_national_system.json', 'w', encoding='utf-8') as f:
    json.dump(kwic_glowbe_national_system, f, ensure_ascii=False, indent=2)

with open(exports_dir / 'kwic_glowbe_industry_standard.json', 'w', encoding='utf-8') as f:
    json.dump(kwic_glowbe_industry_standard, f, ensure_ascii=False, indent=2)

# EcoLexicon exports
kwic_ecolexicon_best_system = _serialize(best_system_kwic_ecolexicon)
kwic_ecolexicon_best_method = _serialize(best_method_kwic_ecolexicon)
kwic_ecolexicon_national_system = _serialize(national_system_kwic_ecolexicon)
kwic_ecolexicon_industry_standard = _serialize(industry_standard_kwic_ecolexicon)

with open(exports_dir / 'kwic_ecolexicon_best_system.json', 'w', encoding='utf-8') as f:
    json.dump(kwic_ecolexicon_best_system, f, ensure_ascii=False, indent=2)

with open(exports_dir / 'kwic_ecolexicon_best_method.json', 'w', encoding='utf-8') as f:
    json.dump(kwic_ecolexicon_best_method, f, ensure_ascii=False, indent=2)

with open(exports_dir / 'kwic_ecolexicon_national_system.json', 'w', encoding='utf-8') as f:
    json.dump(kwic_ecolexicon_national_system, f, ensure_ascii=False, indent=2)

with open(exports_dir / 'kwic_ecolexicon_industry_standard.json', 'w', encoding='utf-8') as f:
    json.dump(kwic_ecolexicon_industry_standard, f, ensure_ascii=False, indent=2)

print(f"✅ Written 12 KWIC JSON files to {exports_dir.resolve()}:")
print("   COCA: best_system, best_method, national_system, industry_standard")
print("   GloWbE: best_system, best_method, national_system, industry_standard")
print("   EcoLexicon: best_system, best_method, national_system, industry_standard")

✅ Written 12 KWIC JSON files to /Users/ejacquot/Documents/Github/getout_of_text_3/examples/public/exports:
   COCA: best_system, best_method, national_system, industry_standard
   GloWbE: best_system, best_method, national_system, industry_standard
   EcoLexicon: best_system, best_method, national_system, industry_standard


____________________________

### Proceed for Minimal Coding Approach

- concordance line annotation
- use the `annotator_app/app.py` to annotate concordance lines interactively